# Análisis de Correlaciones de Spearman

Este notebook realiza el análisis de correlaciones entre las **variables ideológicas** (predictores) y las **variables de cambio de opinión/tiempo** (outcomes) para preparar el análisis SEM.

## Variables Analizadas:

### **Predictores (Variables Ideológicas):**
- `Indice_Progresismo`
- `Indice_Conservadurismo`

### **Outcomes (Variables Dependientes):**

**Grupo 1: Variables Sumadas**
- CO: `Cambio_Op_Sum_Pro_Izq`, `Cambio_Op_Sum_Pro_Der`, `Cambio_Op_Sum_Con_Izq`, `Cambio_Op_Sum_Con_Der`
- CT: `Cambio_Tiempo_Sum_Pro_Izq`, `Cambio_Tiempo_Sum_Pro_Der`, `Cambio_Tiempo_Sum_Con_Izq`, `Cambio_Tiempo_Sum_Con_Der`

**Grupo 2: Variables Filtradas**
- CO: `Cambio_Op_Filt_Pro_Izq`, `Cambio_Op_Filt_Pro_Der`, `Cambio_Op_Filt_Con_Izq`, `Cambio_Op_Filt_Con_Der`
- CT: `Cambio_Tiempo_Filt_Pro_Izq`, `Cambio_Tiempo_Filt_Pro_Der`, `Cambio_Tiempo_Filt_Con_Izq`, `Cambio_Tiempo_Filt_Con_Der`

**Grupo 3: Variables de Congruencia**
- `CO_Congruente`, `CO_Incongruente`
- `CT_Congruente`, `CT_Incongruente`

## Proceso:

1. Calcular correlaciones de Spearman entre todas las variables
2. Generar matrices de correlación
3. Crear heatmaps visuales
4. Guardar resultados en Excel
5. Análisis separado para Generales y Ballotage

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías cargadas exitosamente")

## 1. Cargar Datos

In [ ]:
# Rutas
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

## 2. Definir Variables de Interés

In [ ]:
# Variables predictoras (ideológicas)
Predictores = [
    'Indice_Progresismo',
    'Indice_Conservadurismo'
]

# Variables outcome - Grupo 1: Sumadas
Outcomes_Sumadas = [
    'Cambio_Op_Sum_Pro_Izq',
    'Cambio_Op_Sum_Pro_Der',
    'Cambio_Op_Sum_Con_Izq',
    'Cambio_Op_Sum_Con_Der',
    'Cambio_Tiempo_Sum_Pro_Izq',
    'Cambio_Tiempo_Sum_Pro_Der',
    'Cambio_Tiempo_Sum_Con_Izq',
    'Cambio_Tiempo_Sum_Con_Der'
]

# Variables outcome - Grupo 2: Filtradas
Outcomes_Filtradas = [
    'Cambio_Op_Filt_Pro_Izq',
    'Cambio_Op_Filt_Pro_Der',
    'Cambio_Op_Filt_Con_Izq',
    'Cambio_Op_Filt_Con_Der',
    'Cambio_Tiempo_Filt_Pro_Izq',
    'Cambio_Tiempo_Filt_Pro_Der',
    'Cambio_Tiempo_Filt_Con_Izq',
    'Cambio_Tiempo_Filt_Con_Der'
]

# Variables outcome - Grupo 3: Congruencia
Outcomes_Congruencia = [
    'CO_Congruente',
    'CO_Incongruente',
    'CT_Congruente',
    'CT_Incongruente'
]

# Todas las outcomes juntas
Todas_Outcomes = Outcomes_Sumadas + Outcomes_Filtradas + Outcomes_Congruencia

# Todas las variables para correlación
Todas_Variables = Predictores + Todas_Outcomes

print(f"✓ Variables definidas:")
print(f"  - Predictores: {len(Predictores)}")
print(f"  - Outcomes Sumadas: {len(Outcomes_Sumadas)}")
print(f"  - Outcomes Filtradas: {len(Outcomes_Filtradas)}")
print(f"  - Outcomes Congruencia: {len(Outcomes_Congruencia)}")
print(f"  - Total variables: {len(Todas_Variables)}")

## 3. Función para Calcular Correlaciones de Spearman

In [ ]:
def Calcular_Correlaciones_Spearman(df, variables, nombre_dataset):
    """
    Calcula matriz de correlaciones de Spearman entre variables.
    
    Parámetros:
    -----------
    df : DataFrame
        DataFrame con los datos
    variables : list
        Lista de nombres de variables a correlacionar
    nombre_dataset : str
        Nombre del dataset (para reportes)
    
    Retorna:
    --------
    matriz_corr : DataFrame
        Matriz de correlaciones
    matriz_pvalor : DataFrame
        Matriz de p-valores
    """
    
    print(f"\n{'='*70}")
    print(f"CALCULANDO CORRELACIONES: {nombre_dataset}")
    print(f"{'='*70}")
    
    # Verificar qué variables existen en el DataFrame
    variables_existentes = [v for v in variables if v in df.columns]
    variables_faltantes = [v for v in variables if v not in df.columns]
    
    if variables_faltantes:
        print(f"\n⚠️  Variables no encontradas ({len(variables_faltantes)}):")
        for v in variables_faltantes:
            print(f"  - {v}")
    
    print(f"\n✓ Variables encontradas: {len(variables_existentes)}")
    
    # Seleccionar solo las variables existentes
    df_subset = df[variables_existentes].copy()
    
    # Eliminar filas con NaN
    df_clean = df_subset.dropna()
    
    print(f"✓ Registros válidos: {len(df_clean)} de {len(df)}")
    
    # Calcular matriz de correlaciones
    n_vars = len(variables_existentes)
    matriz_corr = np.zeros((n_vars, n_vars))
    matriz_pvalor = np.zeros((n_vars, n_vars))
    
    print(f"\n🔄 Calculando correlaciones de Spearman...")
    
    for i in range(n_vars):
        for j in range(n_vars):
            if i == j:
                matriz_corr[i, j] = 1.0
                matriz_pvalor[i, j] = 0.0
            else:
                # Calcular correlación de Spearman
                datos_i = df_clean[variables_existentes[i]]
                datos_j = df_clean[variables_existentes[j]]
                
                corr, pval = spearmanr(datos_i, datos_j)
                
                matriz_corr[i, j] = corr
                matriz_pvalor[i, j] = pval
    
    # Convertir a DataFrames
    df_corr = pd.DataFrame(
        matriz_corr,
        index=variables_existentes,
        columns=variables_existentes
    )
    
    df_pvalor = pd.DataFrame(
        matriz_pvalor,
        index=variables_existentes,
        columns=variables_existentes
    )
    
    print(f"✅ Correlaciones calculadas exitosamente")
    
    # Resumen de correlaciones significativas con predictores
    print(f"\n📊 RESUMEN: Correlaciones con Predictores (p < 0.05)")
    print(f"{'-'*70}")
    
    for predictor in Predictores:
        if predictor in variables_existentes:
            print(f"\n{predictor}:")
            
            correlaciones_sig = []
            
            for outcome in Todas_Outcomes:
                if outcome in variables_existentes:
                    corr = df_corr.loc[predictor, outcome]
                    pval = df_pvalor.loc[predictor, outcome]
                    
                    if pval < 0.05:
                        sig = '***' if pval < 0.001 else '**' if pval < 0.01 else '*'
                        correlaciones_sig.append({
                            'Variable': outcome,
                            'r': corr,
                            'p': pval,
                            'sig': sig
                        })
            
            if correlaciones_sig:
                # Ordenar por valor absoluto de correlación
                correlaciones_sig.sort(key=lambda x: abs(x['r']), reverse=True)
                
                for item in correlaciones_sig[:10]:  # Top 10
                    print(f"  {item['Variable']:<35} r={item['r']:>6.3f} (p={item['p']:.4f}) {item['sig']}")
                
                if len(correlaciones_sig) > 10:
                    print(f"  ... y {len(correlaciones_sig) - 10} más")
            else:
                print(f"  ❌ No hay correlaciones significativas")
    
    print(f"\n{'-'*70}")
    
    return df_corr, df_pvalor, variables_existentes

## 4. Función para Crear Heatmap

In [ ]:
def Crear_Heatmap(matriz_corr, titulo, nombre_archivo, carpeta_destino):
    """
    Crea y guarda un heatmap de la matriz de correlaciones.
    
    Parámetros:
    -----------
    matriz_corr : DataFrame
        Matriz de correlaciones
    titulo : str
        Título del gráfico
    nombre_archivo : str
        Nombre del archivo de salida
    carpeta_destino : str
        Carpeta donde guardar el gráfico
    """
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(16, 14))
    
    # Crear heatmap
    sns.heatmap(
        matriz_corr,
        annot=True,
        fmt='.2f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cbar_kws={'label': 'Correlación de Spearman'},
        ax=ax
    )
    
    # Configurar
    ax.set_title(titulo, fontsize=16, fontweight='bold', pad=20)
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    
    # Guardar
    ruta_completa = os.path.join(carpeta_destino, nombre_archivo)
    plt.savefig(ruta_completa, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Heatmap guardado: {nombre_archivo}")

## 5. Función para Guardar en Excel con Formato

In [ ]:
def Guardar_Correlaciones_Excel(df_corr, df_pvalor, nombre_archivo, carpeta_destino):
    """
    Guarda las matrices de correlación y p-valores en Excel con formato.
    
    Parámetros:
    -----------
    df_corr : DataFrame
        Matriz de correlaciones
    df_pvalor : DataFrame
        Matriz de p-valores
    nombre_archivo : str
        Nombre del archivo Excel
    carpeta_destino : str
        Carpeta donde guardar
    """
    
    ruta_completa = os.path.join(carpeta_destino, nombre_archivo)
    
    # Crear archivo Excel con múltiples hojas
    with pd.ExcelWriter(ruta_completa, engine='openpyxl') as writer:
        # Hoja 1: Correlaciones
        df_corr.to_excel(writer, sheet_name='Correlaciones', index=True)
        
        # Hoja 2: P-valores
        df_pvalor.to_excel(writer, sheet_name='P-valores', index=True)
        
        # Hoja 3: Correlaciones con significancia marcada
        df_corr_sig = df_corr.copy()
        
        # Crear DataFrame con texto que incluye significancia
        df_texto = pd.DataFrame(index=df_corr.index, columns=df_corr.columns)
        
        for i in df_corr.index:
            for j in df_corr.columns:
                corr = df_corr.loc[i, j]
                pval = df_pvalor.loc[i, j]
                
                if i == j:
                    df_texto.loc[i, j] = '1.00'
                else:
                    sig = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
                    df_texto.loc[i, j] = f"{corr:.3f}{sig}"
        
        df_texto.to_excel(writer, sheet_name='Correlaciones_Sig', index=True)
    
    # Aplicar formato
    wb = load_workbook(ruta_completa)
    
    # Formato para hoja de correlaciones
    ws_corr = wb['Correlaciones']
    
    # Encabezados en negrita
    for cell in ws_corr[1]:
        cell.font = Font(bold=True)
        cell.alignment = Alignment(horizontal='center', vertical='center')
    
    for cell in ws_corr['A']:
        cell.font = Font(bold=True)
    
    # Centrar valores
    for row in ws_corr.iter_rows(min_row=2, min_col=2):
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Ajustar anchos de columna
    ws_corr.column_dimensions['A'].width = 35
    for col in range(2, ws_corr.max_column + 1):
        ws_corr.column_dimensions[get_column_letter(col)].width = 12
    
    # Guardar
    wb.save(ruta_completa)
    
    print(f"  ✓ Excel guardado: {nombre_archivo}")

## 6. Análisis para GENERALES

In [ ]:
# Calcular correlaciones
df_corr_gen, df_pvalor_gen, vars_gen = Calcular_Correlaciones_Spearman(
    df_Generales,
    Todas_Variables,
    'GENERALES'
)

In [ ]:
# Crear carpeta de resultados
Carpeta_Resultados = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_SEM')
if not os.path.exists(Carpeta_Resultados):
    os.makedirs(Carpeta_Resultados)
    print(f"✓ Carpeta creada: {Carpeta_Resultados}")

# Guardar resultados
print(f"\n📁 Guardando resultados de GENERALES...")

Guardar_Correlaciones_Excel(
    df_corr_gen,
    df_pvalor_gen,
    'Correlaciones_Spearman_Generales.xlsx',
    Carpeta_Resultados
)

Crear_Heatmap(
    df_corr_gen,
    'Correlaciones de Spearman - GENERALES',
    'Heatmap_Correlaciones_Generales.png',
    Carpeta_Resultados
)

## 7. Análisis para BALLOTAGE

In [ ]:
# Calcular correlaciones
df_corr_bal, df_pvalor_bal, vars_bal = Calcular_Correlaciones_Spearman(
    df_Ballotage,
    Todas_Variables,
    'BALLOTAGE'
)

In [ ]:
# Guardar resultados
print(f"\n📁 Guardando resultados de BALLOTAGE...")

Guardar_Correlaciones_Excel(
    df_corr_bal,
    df_pvalor_bal,
    'Correlaciones_Spearman_Ballotage.xlsx',
    Carpeta_Resultados
)

Crear_Heatmap(
    df_corr_bal,
    'Correlaciones de Spearman - BALLOTAGE',
    'Heatmap_Correlaciones_Ballotage.png',
    Carpeta_Resultados
)

## 8. Análisis Comparativo: Predictores vs Outcomes

In [ ]:
print("="*70)
print("ANÁLISIS COMPARATIVO: PREDICTORES VS OUTCOMES")
print("="*70)

# Extraer correlaciones de predictores con outcomes
resultados_comparativos = []

for predictor in Predictores:
    if predictor in vars_gen and predictor in vars_bal:
        print(f"\n📊 {predictor}:")
        print("-"*70)
        
        for outcome in Todas_Outcomes:
            if outcome in vars_gen and outcome in vars_bal:
                corr_gen = df_corr_gen.loc[predictor, outcome]
                pval_gen = df_pvalor_gen.loc[predictor, outcome]
                sig_gen = '***' if pval_gen < 0.001 else '**' if pval_gen < 0.01 else '*' if pval_gen < 0.05 else 'ns'
                
                corr_bal = df_corr_bal.loc[predictor, outcome]
                pval_bal = df_pvalor_bal.loc[predictor, outcome]
                sig_bal = '***' if pval_bal < 0.001 else '**' if pval_bal < 0.01 else '*' if pval_bal < 0.05 else 'ns'
                
                resultados_comparativos.append({
                    'Predictor': predictor,
                    'Outcome': outcome,
                    'Corr_Generales': corr_gen,
                    'p_Generales': pval_gen,
                    'Sig_Generales': sig_gen,
                    'Corr_Ballotage': corr_bal,
                    'p_Ballotage': pval_bal,
                    'Sig_Ballotage': sig_bal,
                    'Diferencia': abs(corr_gen - corr_bal)
                })

df_comparativo = pd.DataFrame(resultados_comparativos)

# Mostrar top correlaciones por predictor
for predictor in Predictores:
    print(f"\n{predictor}:")
    print("  Top 5 correlaciones en Generales:")
    
    df_pred = df_comparativo[df_comparativo['Predictor'] == predictor].copy()
    df_pred_gen = df_pred.sort_values('Corr_Generales', key=abs, ascending=False).head(5)
    
    for _, row in df_pred_gen.iterrows():
        print(f"    {row['Outcome']:<35} r={row['Corr_Generales']:>6.3f} ({row['Sig_Generales']})")
    
    print("\n  Top 5 correlaciones en Ballotage:")
    
    df_pred_bal = df_pred.sort_values('Corr_Ballotage', key=abs, ascending=False).head(5)
    
    for _, row in df_pred_bal.iterrows():
        print(f"    {row['Outcome']:<35} r={row['Corr_Ballotage']:>6.3f} ({row['Sig_Ballotage']})")

print(f"\n{'-'*70}")

# Guardar comparación
ruta_comparacion = os.path.join(Carpeta_Resultados, 'Comparacion_Correlaciones_Generales_vs_Ballotage.xlsx')
df_comparativo.to_excel(ruta_comparacion, index=False)
print(f"\n✓ Comparación guardada: Comparacion_Correlaciones_Generales_vs_Ballotage.xlsx")

## 9. Heatmap Solo de Predictores vs Outcomes

In [ ]:
# Crear heatmaps más enfocados: solo predictores vs outcomes

print("\n📊 Creando heatmaps enfocados (Predictores vs Outcomes)...")

# Para Generales
outcomes_disponibles_gen = [o for o in Todas_Outcomes if o in vars_gen]
predictores_disponibles_gen = [p for p in Predictores if p in vars_gen]

if outcomes_disponibles_gen and predictores_disponibles_gen:
    df_pred_vs_out_gen = df_corr_gen.loc[predictores_disponibles_gen, outcomes_disponibles_gen]
    
    fig, ax = plt.subplots(figsize=(18, 4))
    sns.heatmap(
        df_pred_vs_out_gen,
        annot=True,
        fmt='.3f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        linewidths=0.5,
        cbar_kws={'label': 'Correlación de Spearman'},
        ax=ax
    )
    ax.set_title('Correlaciones: Predictores vs Outcomes - GENERALES', fontsize=14, fontweight='bold', pad=15)
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    
    ruta_heatmap = os.path.join(Carpeta_Resultados, 'Heatmap_Predictores_vs_Outcomes_Generales.png')
    plt.savefig(ruta_heatmap, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  ✓ Heatmap_Predictores_vs_Outcomes_Generales.png")

# Para Ballotage
outcomes_disponibles_bal = [o for o in Todas_Outcomes if o in vars_bal]
predictores_disponibles_bal = [p for p in Predictores if p in vars_bal]

if outcomes_disponibles_bal and predictores_disponibles_bal:
    df_pred_vs_out_bal = df_corr_bal.loc[predictores_disponibles_bal, outcomes_disponibles_bal]
    
    fig, ax = plt.subplots(figsize=(18, 4))
    sns.heatmap(
        df_pred_vs_out_bal,
        annot=True,
        fmt='.3f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        linewidths=0.5,
        cbar_kws={'label': 'Correlación de Spearman'},
        ax=ax
    )
    ax.set_title('Correlaciones: Predictores vs Outcomes - BALLOTAGE', fontsize=14, fontweight='bold', pad=15)
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    
    ruta_heatmap = os.path.join(Carpeta_Resultados, 'Heatmap_Predictores_vs_Outcomes_Ballotage.png')
    plt.savefig(ruta_heatmap, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  ✓ Heatmap_Predictores_vs_Outcomes_Ballotage.png")

## 10. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN FINAL: ANÁLISIS DE CORRELACIONES DE SPEARMAN")
print("="*70)

print("\n📊 Variables analizadas:")
print(f"  - Predictores: {len(Predictores)} ({', '.join(Predictores)})")
print(f"  - Outcomes Sumadas: {len(Outcomes_Sumadas)}")
print(f"  - Outcomes Filtradas: {len(Outcomes_Filtradas)}")
print(f"  - Outcomes Congruencia: {len(Outcomes_Congruencia)}")
print(f"  - Total Outcomes: {len(Todas_Outcomes)}")

print("\n📁 Archivos generados:")
print("  Generales:")
print("    - Correlaciones_Spearman_Generales.xlsx")
print("    - Heatmap_Correlaciones_Generales.png")
print("    - Heatmap_Predictores_vs_Outcomes_Generales.png")
print("\n  Ballotage:")
print("    - Correlaciones_Spearman_Ballotage.xlsx")
print("    - Heatmap_Correlaciones_Ballotage.png")
print("    - Heatmap_Predictores_vs_Outcomes_Ballotage.png")
print("\n  Comparativo:")
print("    - Comparacion_Correlaciones_Generales_vs_Ballotage.xlsx")

print("\n🎯 Próximos pasos:")
print("  1. Revisar correlaciones significativas")
print("  2. Identificar patrones entre elecciones")
print("  3. Preparar para modelos SEM (Notebooks 55-58)")

print("\n" + "="*70)
print("✅ ANÁLISIS DE CORRELACIONES COMPLETADO")
print("="*70)